Implementation of GPD using SciPy

In [1]:
import math
import numpy as np
import pandas as pd
import scipy
import seaborn as sn
sn.set(rc={'figure.figsize':(11, 4)})
import scikits
from scipy import stats, optimize
from scipy.stats import genpareto
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats
from sklearn import svm



%matplotlib notebook

In [2]:
data = pd.read_csv('study_case1_prms_input.csv')

In [3]:
data

,runoff_obs,basin_cfs_pred,Id,year,month,day,precip,tmax,tmin
0,2.7,3.890482,1,2002,10,1,0.13,57.4,34.6
1,2.7,3.041588,2,2002,10,2,0.97,38.4,29.7
2,2.8,2.796749,3,2002,10,3,0.04,41.2,31.7
3,2.8,2.600715,4,2002,10,4,0.00,52.9,34.0
4,2.6,2.439560,5,2002,10,5,0.00,50.5,34.8
...,...,...,...,...,...,...,...,...,...
3648,4.4,5.014700,3649,2012,9,26,0.00,59.2,41.0
3649,4.3,4.979207,3650,2012,9,27,0.00,64.1,43.2
3650,4.2,5.001390,3651,2012,9,28,0.00,68.0,46.2
3651,4.2,5.114594,3652,2012,9,29,0.00,69.8,50.9


In [4]:
info = data.info()
        
print(info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 9 columns):
runoff_obs        3653 non-null float64
basin_cfs_pred    3653 non-null float64
Id                3653 non-null int64
year              3653 non-null int64
month             3653 non-null int64
day               3653 non-null int64
precip            3653 non-null float64
tmax              3653 non-null float64
tmin              3653 non-null float64
dtypes: float64(5), int64(4)
memory usage: 257.0 KB
None


In [5]:
from datetime import date

In [6]:
data = data.assign(Date=pd.to_datetime(data[['year', 'month','day']]))

In [7]:
data

,runoff_obs,basin_cfs_pred,Id,year,month,day,precip,tmax,tmin,Date
0,2.7,3.890482,1,2002,10,1,0.13,57.4,34.6,2002-10-01
1,2.7,3.041588,2,2002,10,2,0.97,38.4,29.7,2002-10-02
2,2.8,2.796749,3,2002,10,3,0.04,41.2,31.7,2002-10-03
3,2.8,2.600715,4,2002,10,4,0.00,52.9,34.0,2002-10-04
4,2.6,2.439560,5,2002,10,5,0.00,50.5,34.8,2002-10-05
...,...,...,...,...,...,...,...,...,...,...
3648,4.4,5.014700,3649,2012,9,26,0.00,59.2,41.0,2012-09-26
3649,4.3,4.979207,3650,2012,9,27,0.00,64.1,43.2,2012-09-27
3650,4.2,5.001390,3651,2012,9,28,0.00,68.0,46.2,2012-09-28
3651,4.2,5.114594,3652,2012,9,29,0.00,69.8,50.9,2012-09-29


In [8]:
data.drop(data.columns[[3,4,5]], axis=1, inplace=True)

In [9]:
data

,runoff_obs,basin_cfs_pred,Id,precip,tmax,tmin,Date
0,2.7,3.890482,1,0.13,57.4,34.6,2002-10-01
1,2.7,3.041588,2,0.97,38.4,29.7,2002-10-02
2,2.8,2.796749,3,0.04,41.2,31.7,2002-10-03
3,2.8,2.600715,4,0.00,52.9,34.0,2002-10-04
4,2.6,2.439560,5,0.00,50.5,34.8,2002-10-05
...,...,...,...,...,...,...,...
3648,4.4,5.014700,3649,0.00,59.2,41.0,2012-09-26
3649,4.3,4.979207,3650,0.00,64.1,43.2,2012-09-27
3650,4.2,5.001390,3651,0.00,68.0,46.2,2012-09-28
3651,4.2,5.114594,3652,0.00,69.8,50.9,2012-09-29


In [10]:
data = data.set_index('Date')

In [11]:
data

,runoff_obs,basin_cfs_pred,Id,precip,tmax,tmin
Date,,,,,,
2002-10-01,2.7,3.890482,1,0.13,57.4,34.6
2002-10-02,2.7,3.041588,2,0.97,38.4,29.7
2002-10-03,2.8,2.796749,3,0.04,41.2,31.7
2002-10-04,2.8,2.600715,4,0.00,52.9,34.0
2002-10-05,2.6,2.439560,5,0.00,50.5,34.8
...,...,...,...,...,...,...
2012-09-26,4.4,5.014700,3649,0.00,59.2,41.0
2012-09-27,4.3,4.979207,3650,0.00,64.1,43.2
2012-09-28,4.2,5.001390,3651,0.00,68.0,46.2


In [12]:
from scipy.stats import pareto
pareto.fit(data['runoff_obs'])

C:\Users\mattar18\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2316: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu


(1.0286139226046471, -1.2283150748012654, 1.6983150582976179)

In [13]:
#fit svr model for the whole dataset
data.drop(data.columns[[1]], axis=1, inplace=True)

In [14]:
data

,runoff_obs,Id,precip,tmax,tmin
Date,,,,,
2002-10-01,2.7,1,0.13,57.4,34.6
2002-10-02,2.7,2,0.97,38.4,29.7
2002-10-03,2.8,3,0.04,41.2,31.7
2002-10-04,2.8,4,0.00,52.9,34.0
2002-10-05,2.6,5,0.00,50.5,34.8
...,...,...,...,...,...
2012-09-26,4.4,3649,0.00,59.2,41.0
2012-09-27,4.3,3650,0.00,64.1,43.2
2012-09-28,4.2,3651,0.00,68.0,46.2


In [15]:
data_values = data.values
features = data_values[:,1:]
values = data_values[:,0]

In [16]:
pip install sklearn-extensions


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install git+https://github.com/OpenHydrology/lmoments3.git

  Cloning https://github.com/OpenHydrology/lmoments3.git to c:\users\mattar18\appdata\local\temp\pip-req-build-q19mnike
  Created wheel for lmoments3: filename=lmoments3-1.0.4+6.gd2f3073-cp36-none-any.whl size=29447 sha256=857f8395d9aa565c37c3190b6138b24bcde4a3be62a452fd004f15d403c787da
  Stored in directory: C:\Users\mattar18\AppData\Local\Temp\pip-ephem-wheel-cache-v_twun_a\wheels\71\f6\66\978da9ec05dd420fc63f25af47ce80626ccfeae757861c5436
Successfully built lmoments3
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/OpenHydrology/lmoments3.git 'C:\Users\mattar18\AppData\Local\Temp\pip-req-build-q19mnike'


In [18]:
pip3 install scikit-extremes

Note: you may need to restart the kernel to use updated packages.


  ERROR: Could not find a version that satisfies the requirement scikit-extremes (from versions: none)
ERROR: No matching distribution found for scikit-extremes


In [19]:
    
from setuptools import (
    setup,
    find_packages,
)  

install_requires = ["numpy", "matplotlib", "scipy>=1.0", "numdifftools>=0.9.20"]

extras_require = {
    "dev": ["pytest", "pytest-cov", "nbsphinx", "sphinx", "sphinx_rtd_theme"]
}

In [20]:
cd scikit-extremes

[WinError 2] The system cannot find the file specified: 'scikit-extremes'
C:\Users\mattar18\RA


In [21]:
pip install -e .

Note: you may need to restart the kernel to use updated packages.


ERROR: File "setup.py" not found. Directory cannot be installed in editable mode: C:\Users\mattar18\RA
